# First attempt to scraping the legislative proposals

## 1. Library and data import

In [28]:
import requests
from bs4 import NavigableString, BeautifulSoup
import pandas as pd
import ast
import re

In [29]:
# Load the manipulated data
df = pd.read_csv('final_sample_cod_manip_2025.csv')
pd.set_option('display.max_columns', None)

In [30]:
# Display the data
df

,Unnamed: 0,url,reference,title,subjects,key_players,key_events,documentation_gateway,transparency,final_act,is_legislative_proposal,Procedure reference,Procedure type,Procedure subtype,Legislative instrument,Amendments and repeals,Legal basis,Stage reached in procedure,Committee dossier,Mandatory consultation of other institutions,Other legal basis
0,0,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0255(COD),Justice programme 2028-2034,7.40.02 Judicial cooperation in civil and comm...,{'European Parliament': [{'Joint committee res...,"{'03/09/2025': [{'Date': '03/09/2025', 'Event'...","[{'Institution': 'European Commission', 'Docum...",NaN,NaN,False,2025/0255(COD),COD - Ordinary legislative procedure (ex-codec...,Legislation,Regulation,"[{'text': 'Repealing Regulation 2021/693'}, {'...",Rules of Procedure EP 58 Treaty on the Functio...,Awaiting committee decision,CJ03/10/03804,NaN,NaN
1,1,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0405(COD),Placing on the market of genetically modified ...,"3.10.09.06 Agro-genetics, GMOs, 4.60.02 Consum...",{'European Parliament': [{'Joint committee res...,"{'16/12/2025': [{'Date': '16/12/2025', 'Event'...","[{'Institution': 'European Commission', 'Docum...",NaN,NaN,True,2025/0405(COD),COD - Ordinary legislative procedure (ex-codec...,Legislation,Directive,"[{'text': 'Amending Directive 2001/18/EC'}, {'...",Treaty on the Functioning of the EU TFEU 114 T...,Preparatory phase in Parliament,NaN,[{'text': 'European Economic and Social Commit...,NaN
2,2,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0540(COD),"Union support for asylum, migration and integr...",7.10 Free movement and integration of third-co...,{'European Parliament': [{'Committee responsib...,"{'16/07/2025': [{'Date': '16/07/2025', 'Event'...","[{'Institution': 'European Commission', 'Docum...",{'Other Members': [{'Name': [{'text': 'ASENS L...,NaN,True,2025/0540(COD),COD - Ordinary legislative procedure (ex-codec...,Legislation,Regulation,NaN,Rules of Procedure EP 58 Treaty on the Functio...,Awaiting committee decision,LIBE/10/03652,NaN,NaN
3,3,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0550(COD),'AgoraEU' programme 2028–2034,"1.20 Citizen's rights, 4.10.03 Child protectio...",{'European Parliament': [{'Joint committee res...,"{'16/07/2025': [{'Date': '16/07/2025', 'Event'...","[{'Institution': 'European Commission', 'Docum...","{'Rapporteurs, Shadow Rapporteurs and Committe...",NaN,True,2025/0550(COD),COD - Ordinary legislative procedure (ex-codec...,Legislation,Regulation,"[{'text': 'Repealing Regulation 2021/818'}, {'...",Rules of Procedure EP 58 Rules of Procedure EP...,Awaiting committee decision,CJ65/10/03702,[{'text': 'European Economic and Social Commit...,NaN
4,4,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0227(COD),Global Europe,"6.20 Common commercial policy in general, 8.70...",{'European Parliament': [{'Joint committee res...,"{'16/07/2025': [{'Date': '16/07/2025', 'Event'...","[{'Institution': 'European Commission', 'Docum...",NaN,NaN,True,2025/0227(COD),COD - Ordinary legislative procedure (ex-codec...,Legislation,Regulation,NaN,Rules of Procedure EP 58 Rules of Procedure EP...,Awaiting committee decision,CJ19/10/03697,"[{'text': 'European Court of Auditors', 'href'...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,114,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0232(COD),Protection of workers from the risks related t...,"4.15.15 Health and safety at work, occupationa...",{'European Parliament': [{'Committee responsib...,"{'18/07/2025': [{'Date': '18/07/2025', 'Event'...","[{'Institution': 'European Commission', 'Docum...","{'Rapporteurs, Shadow Rapporteurs and Committe...",NaN,True,2025/0232(COD),COD - Ordinary legislative procedure (ex-codec...,Legislation,Directive,"[{'text': 'Amending Directive 2004/37/EC'}, {'...",Treaty on the Functioning of the EU TFEU 153-p...,Awaiting committee decision,EMPL/10/037

## 2. Function definition

In [ ]:
def get_latest_ec_legislative_proposal(doc_list):
    """
    Returns the CELEX link of the most recent final legislative proposal
    of the European Commission.
    """
    try:
        doc_list = ast.literal_eval(doc_list) if isinstance(doc_list, str) else doc_list
    except Exception:
        return None

    if not isinstance(doc_list, list):
        return None

    # List to store all final references
    final_refs = []

    for doc in doc_list:
        if isinstance(doc, dict) and doc.get('Institution') == 'European Commission' and doc.get('Document type') == 'Legislative proposal':
            ref = doc.get('Reference')
            # Convert ref to a list if it is a multiple string
            try:
                ref_list = ast.literal_eval("[" + ref + "]") if isinstance(ref, str) else (ref if isinstance(ref, list) else [ref])
            except Exception:
                continue

            for r in ref_list:
                href = r.get('href') if isinstance(r, dict) else None
                if href and 'COMfinal' in href:
                    final_refs.append(href)

    if not final_refs:
        return None

    # Get the latest final href (the most recent one)
    latest_href = final_refs[-1]

    # Optional: transform into a CELEX link if an_doc is present
    match = re.search(r"an_doc=(\d+)", latest_href)
    if match:
        year = match.group(1)
        number = latest_href[-4:]
        if number and not number[0].isdigit():
            number = '0' + number[1:]
        return f"https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:5{year}PC{number}"

    return latest_href

In [32]:
# Scraper function: fetches a webpage and returns its BeautifulSoup object
def fetch_soup(url):
    try:
        # Send an HTTP GET request to the given URL with a timeout of 10 seconds
        r = requests.get(url, timeout=10)
        # Raise an HTTPError if the response code indicates an error (4xx or 5xx)
        r.raise_for_status()
        # Return the parsed HTML content as a BeautifulSoup object
        return BeautifulSoup(r.text, "html.parser")
    except requests.exceptions.HTTPError as e:
        # Print detailed error message for HTTP-related issues
        print(f"HTTP error for {url}: {e}")
    except requests.exceptions.RequestException as e:
        # Catch any other network-related or timeout exceptions
        print(f"Request failed for {url}: {e}")
    # Return None if any error occurred
    return None


# Function that parses an HTML table element and extracts its structure
def parse_table(table_el):
    """Extract table rows and detect headers."""
    # Check if the table has any header cells (<th>)
    has_header = bool(table_el.find_all("th"))
    rows = []
    # Loop through each table row
    for tr in table_el.find_all("tr"):
        # Extract text from all header and data cells
        cells = [c.get_text(" ", strip=True) for c in tr.find_all(["th", "td"])]
        # Only add non-empty rows
        if any(cells):
            rows.append(cells)
    # Return whether the table has headers, and the list of rows
    return has_header, rows


# Function that converts a parsed table into a format that is readable
def table_to_narrative(has_header, rows):
    """Convert table into readable narrative text."""
    lines = []
    # Return empty string if no rows found
    if not rows:
        return ""
    # Identify header row (if any)
    header = rows[0] if has_header else None
    # Define body as all rows after the header
    body = rows[1:] if header else rows

    # Add an introductory line mentioning column headers (if present)
    if header:
        lines.append("Table with columns: " + ", ".join(header))
    else:
        lines.append("Table:")

    # Loop through each row and describe it in sentence-like form
    for i, row in enumerate(body, start=1):
        # If headers exist and row length matches header length, label each value with its header
        if header and len(row) == len(header):
            row_text = "; ".join(f"{h}: {v}" for h, v in zip(header, row))
        else:
            # Otherwise, just join values without labels
            row_text = "; ".join(row)
        # Add formatted row text to the narrative
        lines.append(f"Row {i} — {row_text}")
    # Return the full narrative text as a string
    return "\n".join(lines)


# Function to extract both text and tables from an HTML page, as structured blocks
def extract_blocks(soup):
    # Define which HTML tags to extract (text and table tags)
    text_tags = ("p", "h1", "h2", "h3", "h4", "h5", "h6")
    # Find all paragraphs, headers, and tables in the HTML
    elems = soup.find_all(list(text_tags) + ["table"])
    blocks = []
    # Iterate through each found element
    for el in elems:
        if el.name == "table":
            # Parse the table and convert it into readable text
            has_header, rows = parse_table(el)
            if rows:
                blocks.append({"type": "table", "text": table_to_narrative(has_header, rows)})
        else:
            # Skip text that is nested inside a table (to avoid duplicates)
            if el.find_parent("table"):
                continue
            # Extract the text content from the element, preserving spaces
            txt = el.get_text(separator=" ", strip=True)
            # Only add non-empty text blocks
            if txt:
                blocks.append({"type": "text", "text": txt})
    # Return a list of extracted content blocks (text or tables)
    return blocks

# Function that merges all extracted blocks into one continuous string
def flatten_blocks(blocks):
    # Join the text content of all blocks, separating them with double newlines
    return "\n\n".join(b["text"] for b in blocks)

## 3. Data manipulation and scraping

In [33]:
# Get the legislative proposal's url
df['legislative_proposal_url'] = df['documentation_gateway'].apply(get_latest_ec_legislative_proposal)

In [35]:
# List to store non managed urls
lp_url_not_found = []
df['legislative_proposal_text'] = None

# Scraping the content of the legislative proposals
for idx, row in df.iterrows():
    print(f"Processing row {idx} out of {len(df)}")
    if row['legislative_proposal_url'] is None :
        lp_url_not_found.append(url)
        continue  # skip broken or missing pages
    else :
        url = row['legislative_proposal_url'] 
        soup = fetch_soup(url)    
        blocks = extract_blocks(soup)
        raw_text = flatten_blocks(blocks).replace('\n', ' ')
        df.loc[idx, 'legislative_proposal_text'] = raw_text   

Processing row 0 out of 119
Processing row 1 out of 119
Processing row 2 out of 119
Processing row 3 out of 119
Processing row 4 out of 119
Processing row 5 out of 119
Processing row 6 out of 119
Processing row 7 out of 119
Processing row 8 out of 119
Processing row 9 out of 119
Processing row 10 out of 119
Processing row 11 out of 119
Processing row 12 out of 119
Processing row 13 out of 119
Processing row 14 out of 119
Processing row 15 out of 119
Processing row 16 out of 119
Processing row 17 out of 119
Processing row 18 out of 119
Processing row 19 out of 119
Processing row 20 out of 119
Processing row 21 out of 119
Processing row 22 out of 119
Processing row 23 out of 119
Processing row 24 out of 119
Processing row 25 out of 119
Processing row 26 out of 119
Processing row 27 out of 119
Processing row 28 out of 119
Processing row 29 out of 119
Processing row 30 out of 119
Processing row 31 out of 119
Processing row 32 out of 119
Processing row 33 out of 119
Processing row 34 out of

In [36]:
# Display the data
df

,Unnamed: 0,url,reference,title,subjects,key_players,key_events,documentation_gateway,transparency,final_act,is_legislative_proposal,Procedure reference,Procedure type,Procedure subtype,Legislative instrument,Amendments and repeals,Legal basis,Stage reached in procedure,Committee dossier,Mandatory consultation of other institutions,Other legal basis,legislative_proposal_url,legislative_proposal_text
0,0,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0255(COD),Justice programme 2028-2034,7.40.02 Judicial cooperation in civil and comm...,{'European Parliament': [{'Joint committee res...,"{'03/09/2025': [{'Date': '03/09/2025', 'Event'...","[{'Institution': 'European Commission', 'Docum...",NaN,NaN,False,2025/0255(COD),COD - Ordinary legislative procedure (ex-codec...,Legislation,Regulation,"[{'text': 'Repealing Regulation 2021/693'}, {'...",Rules of Procedure EP 58 Treaty on the Functio...,Awaiting committee decision,CJ03/10/03804,NaN,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,"EUROPEAN COMMISSION Brussels, 3.9.2025 COM(2..."
1,1,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0405(COD),Placing on the market of genetically modified ...,"3.10.09.06 Agro-genetics, GMOs, 4.60.02 Consum...",{'European Parliament': [{'Joint committee res...,"{'16/12/2025': [{'Date': '16/12/2025', 'Event'...","[{'Institution': 'European Commission', 'Docum...",NaN,NaN,True,2025/0405(COD),COD - Ordinary legislative procedure (ex-codec...,Legislation,Directive,"[{'text': 'Amending Directive 2001/18/EC'}, {'...",Treaty on the Functioning of the EU TFEU 114 T...,Preparatory phase in Parliament,NaN,[{'text': 'European Economic and Social Commit...,NaN,None,None
2,2,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0540(COD),"Union support for asylum, migration and integr...",7.10 Free movement and integration of third-co...,{'European Parliament': [{'Committee responsib...,"{'16/07/2025': [{'Date': '16/07/2025', 'Event'...","[{'Institution': 'European Commission', 'Docum...",{'Other Members': [{'Name': [{'text': 'ASENS L...,NaN,True,2025/0540(COD),COD - Ordinary legislative procedure (ex-codec...,Legislation,Regulation,NaN,Rules of Procedure EP 58 Treaty on the Functio...,Awaiting committee decision,LIBE/10/03652,NaN,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,"EUROPEAN COMMISSION Brussels, 16.7.2025 COM(..."
3,3,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0550(COD),'AgoraEU' programme 2028–2034,"1.20 Citizen's rights, 4.10.03 Child protectio...",{'European Parliament': [{'Joint committee res...,"{'16/07/2025': [{'Date': '16/07/2025', 'Event'...","[{'Institution': 'European Commission', 'Docum...","{'Rapporteurs, Shadow Rapporteurs and Committe...",NaN,True,2025/0550(COD),COD - Ordinary legislative procedure (ex-codec...,Legislation,Regulation,"[{'text': 'Repealing Regulation 2021/818'}, {'...",Rules of Procedure EP 58 Rules of Procedure EP...,Awaiting committee decision,CJ65/10/03702,[{'text': 'European Economic and Social Commit...,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,"EUROPEAN COMMISSION Brussels, 16.7.2025 COM(..."
4,4,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0227(COD),Global Europe,"6.20 Common commercial policy in general, 8.70...",{'European Parliament': [{'Joint committee res...,"{'16/07/2025': [{'Date': '16/07/2025', 'Event'...","[{'Institution': 'European Commission', 'Docum...",NaN,NaN,True,2025/0227(COD),COD - Ordinary legislative procedure (ex-codec...,Legislation,Regulation,NaN,Rules of Procedure EP 58 Rules of Procedure EP...,Awaiting committee decision,CJ19/10/03697,"[{'text': 'European Court of Auditors', 'href'...",NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,"EUROPEAN COMMISSION Brussels, 16.7.2025 COM(..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,114,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0232(COD),Protection of workers from the risks related t...,"4.15.15 Health and safety

In [37]:
# Saving the non managed urls to identify them and manipulate them later
df_urls = pd.DataFrame([[lp_url_not_found]], columns=["urls_not_found"])
df_urls.to_csv('lp_url_not_found_2025.csv')

In [38]:
# Saving the scraped data
df.to_csv('final_sample_cod_legislative_scrape_2025.csv')